# Overrall
* Istio helps reduce the complexity of deployments, and eases the strain on your development teams. Istio’s diverse feature set lets you successfully, and efficiently, run a distributed microservice architecture, and provides a uniform way to secure, connect, and monitor microservices.
* The term service mesh is used to describe the network of microservices that make up such applications and the interactions between them. As a service mesh grows in size and complexity, it can become harder to understand and manage. Its requirements can include `discovery, load balancing, failure recovery, metrics, and monitoring`. A service mesh also often has more complex operational requirements, like `A/B testing, canary releases, rate limiting, access control, and end-to-end authentication`.
* Istio support to services by deploying a special sidecar proxy throughout your environment that intercepts all network communication between microservices, then configure and manage Istio using its control plane functionality, which includes
    * Automatic load balancing for HTTP, gRPC, WebSocket, and TCP traffic.
    * Fine-grained control of traffic behavior with rich routing rules, retries, failovers, and fault injection.
    * A pluggable policy layer and configuration API supporting access controls, rate limits and quotas.
    * Automatic metrics, logs, and traces for all traffic within a cluster, including cluster ingress and egress.
    * Secure service-to-service communication in a cluster with strong identity-based authentication and authorization.
    
### Core features
* Traffic management: 
    * Istio easy rules configuration and traffic routing lets you control the flow of traffic and API calls between services. 
    * Istio simplifies `configuration of service-level properties like circuit breakers, timeouts, and retries`, and makes it a breeze to set up important tasks like A/B testing, canary rollouts, and staged rollouts with percentage-based traffic splits.
* Security:
    * Istio security capabilities free developers to focus on security at the application level. 
    * Istio provides the underlying secure communication channel, and manages authentication, authorization, and encryption of service communication at scale.
    * Istio, service communications are secured by default, letting you enforce policies consistently across diverse protocols and runtimes.
* Observability:
    * Istio robust tracing, monitoring, and logging give you deep insights into your service mesh deployment. Gain a real understanding of how service performance impacts things upstream and downstream with Istio’s monitoring features.
* Platform support:
    * Service deployment on Kubernetes
    * Services registered with Consul
    * Services running on individual virtual machines
    
### Architecture
* Istio service mesh is logically split into a data plane and a control plane
    * The data plane is composed of a set of intelligent proxies (Envoy) deployed as sidecars. These proxies mediate and control all network communication between microservices along with Mixer.
    * The control plane manages and configures the proxies to route traffic. Additionally, the control plane configures Mixers to enforce policies and collect telemetry.
    * <img src="../../images/javaee/istio-arch.svg" width="450px">
* <b>Envoy</b>
    * Istio uses an extended version of the Envoy proxy. Envoy is a high-performance proxy developed in C++ to mediate all inbound and outbound traffic for all services in the service mesh. 
    * Envoy is deployed as a sidecar to the relevant service in the same Kubernetes pod. This deployment allows Istio to extract a wealth of signals about traffic behavior as attributes. Istio can use these attributes in Mixer to enforce policy decisions, and send them to monitoring systems to provide information about the behavior of the entire mesh.
* <b>Mixer</b> is a platform-independent component. Mixer enforces access control and usage policies across the service mesh, and collects telemetry data from the Envoy proxy and other services. The proxy extracts request level attributes, and sends them to Mixer for evaluation. 
* <b>Pilot</b> provides service discovery for the Envoy sidecars, traffic management capabilities for intelligent routing (e.g., `A/B tests, canary deployments`, etc.), and resiliency (`timeouts, retries, circuit breakers`, etc.). Pilot converts high level routing rules that control traffic behavior into Envoy-specific configurations, and propagates them to the sidecars at runtime. 
* <b>Citadel</b> provides strong service-to-service and end-user authentication with built-in identity and credential management.
* <b>Galley</b> validates user authored Istio API configuration on behalf of the other Istio control plane components. Over time, Galley will take over responsibility as the top-level configuration ingestion, processing and distribution component of Istio. It will be responsible for insulating the rest of the Istio components from the details of obtaining user configuration from the underlying platform.

### Design Goals
* <b>Maximize Transparency</b>: Istio can automatically inject itself into all the network paths between services. Istio uses sidecar proxies to capture traffic and, where possible, automatically program the networking layer to route traffic through those proxies without any changes to the deployed application code.
* <b>Incrementality</b>: As operators and developers become more dependent on the functionality that Istio provides, the system must grow with their needs. While we continue to add new features, the greatest need is the ability to extend the policy system, to integrate with other sources of policy and control, and to propagate signals about mesh behavior to other systems for analysis. The policy runtime supports a standard extension mechanism for plugging in other services. 
* <b>Portability</b>: Istio must run on any cloud or on-premises environment with minimal effort. The task of porting Istio-based services to new environments must be trivial. Using Istio, you are able to operate a single service deployed into multiple environments.  
* <b>Policy Uniformity</b>: Istio maintains the policy system as a distinct service with its own API rather than the policy system being baked into the proxy sidecar, allowing services to directly integrate with it as needed.

### Playaround
* <b>Quick Start</b>
    * https://istio.io/docs/setup/kubernetes/
    * https://istio.io/docs/setup/consul/
* <b>Application example operation</b>: https://istio.io/docs/examples/bookinfo/
* <b>Configuration reference</b>: https://istio.io/docs/reference/config/
* <b>Commands reference</b>:https://istio.io/docs/reference/commands/

****

# Traffic Management
* Istio provides a variety of traffic management features that live outside the application code. As well as <b>dynamic request routing for A/B testing</b>, <b>gradual rollouts</b>, and <b>canary releases</b>, it also <b>handles failure recovery</b> using <b>timeouts</b>, <b>retries</b>, <b>circuit breakers</b>, and <b>fault injection</b> to test the compatibility of failure recovery policies across services. 
* <b>Pilot and Envoy</b>
    * <b>Pilot</b> is the core component used for traffic management in Istio, which manages and configures all the Envoy proxy instances deployed in a particular Istio service mesh. 
    * <b>Pilot</b> lets you specify which rules you want to use to route traffic between Envoy proxies and configure failure recovery features such as timeouts, retries, and circuit breakers. It also maintains a canonical model of all the services in the mesh and uses this model to let Envoy instances know about the other Envoy instances in the mesh via its discovery service. 
    * <b>Pilot</b> enables `service discovery, dynamic updates to load balancing pools and routing tables`.
    * <b>Pilot</b> maintains a `canonical representation of services` in the mesh that is independent of the underlying platform. Platform-specific adapters in Pilot are responsible for populating this canonical model appropriately. An Envoy-specific configuration is then generated based on the canonical representation.
    * <img src="../../images/javaee/PilotAdapters.svg" width="450px">
    * <b>Envoy</b> instance maintains load balancing information based on the information it gets from Pilot and periodic health-checks of other instances in its load-balancing pool, allowing it to intelligently distribute traffic between destination instances while following its specified routing rules.
* <b>Request routing</b>
    * Istio introduces the concept of a service version, which is a finer-grained way to subdivide service instances by versions (v1, v2) or environment (staging, prod). These variants are not necessarily different API versions: they could be iterative changes to the same service, deployed in different environments (prod, staging, dev, etc.). Common scenarios where this is used include A/B testing or canary rollouts. 
    * <b>Communication between services</b>
        * Envoy determines its actual choice of service version dynamically based on the routing rules that you specify by using Pilot. Clients of a service have no knowledge of different versions of the service. They can continue to access the services using the hostname/IP address of the service. The Envoy sidecar/proxy intercepts and forwards all requests/responses between the client and the service.
        * <img src="../../images/javaee/ServiceModel_Versions.svg" width="350px">
    * <b>Ingress and egress</b>
        * Istio assumes that all traffic entering and leaving the service mesh transits through Envoy proxies. By deploying an Envoy proxy in front of services, you can conduct A/B testing, deploy canary services, etc. for user-facing services. Similarly, by routing traffic to external web services via the Envoy sidecar, you can add failure recovery features such as timeouts, retries, and circuit breakers and obtain detailed metrics on the connections to these services.
        * <img src="../../images/javaee/ServiceModel_RequestFlow.svg" width="450px">
* <b>Discovery and load balancing</b>
    * <b>Pilot</b> consumes information from the service registry and provides a platform-independent service discovery interface. 
    * <b>Envoy</b> instances in the mesh perform service discovery and dynamically update their load balancing pools accordingly.
    * <b>Envoy</b> distributes the traffic across instances in the load balancing pool. While Envoy supports several sophisticated load balancing algorithms, Istio currently allows three load balancing modes: `round robin, random, and weighted least request`.
    * <b>Envoy</b> periodically checks the health of each instance in the pool. Envoy follows a circuit breaker pattern to classify instances as unhealthy or healthy based on their failure rates for the health check API call. 
    * <img src="../../images/javaee/LoadBalancing.svg" width="350px">
* <b>Handling failures</b>
    * Timeouts
    * Bounded retries with timeout budgets and variable jitter between retries
    * Limits on number of concurrent connections and requests to upstream services
    * Active (periodic) health checks on each member of the load balancing pool
    * Fine-grained circuit breakers (passive health checks) – applied per instance in the load balancing pool
* <b><i>Rule configuration</i></b>: https://istio.io/docs/concepts/traffic-management/#rule-configuration
    * <b>VirtualService configuration</b> defines the rules that control how requests for a service are routed within an Istio service mesh.
    * <b>DestinationRule configures</b> the set of policies to be applied to a request after VirtualService routing has occurred.
    * <b> ServiceEntry configuration</b> is commonly used to enable requests to services outside of an Istio service mesh.
    * <b> Gateway configures</b> a load balancer for HTTP/TCP traffic, most commonly operating at the edge of the mesh to enable ingress traffic for an application.
* <b><i>Operations</i></b>: https://istio.io/docs/tasks/traffic-management/

****

# Security
* The Istio security features provide strong identity, powerful policy, transparent TLS encryption, and authentication, authorization and audit (AAA) tools to protect your services and data. The goals of Istio security are:
    * Security by default: no changes needed for application code and infrastructure
    * Defense in depth: integrate with existing security systems to provide multiple layers of defense
    * Zero-trust network: build security solutions on untrusted networks
* <b>High-level architecture</b>
    * <b>Citadel</b> for key and certificate management
    * <b>Sidecar</b> and perimeter proxies to implement secure communication between clients and servers
    * <b>Pilot</b> to distribute authentication policies and secure naming information to the proxies
    * <b>Mixer</b> to manage authorization and auditing
    * <img src="../../images/javaee/security-architecture.svg" width="600px">
* <b>Authentication</b>
    * Transport authentication, also known as service-to-service authentication: verifies the direct client making the connection. Istio offers mutual TLS as a full stack solution for transport authentication.
        * Provides each service with a strong identity representing its role to enable interoperability across clusters and clouds.
        * Secures service-to-service communication and end-user-to-service communication.
        * Provides a key management system to automate key and certificate generation, distribution, and rotation.
        * <b>Mutual TLS authentication</b>: https://istio.io/docs/concepts/security/#mutual-tls-authentication
    * Origin authentication, also known as end-user authentication: verifies the original client making the request as an end-user or device. Istio enables request-level authentication with JSON Web Token (JWT) validation and a streamlined developer experience for Auth0, Firebase Auth, Google Auth, and custom auth.
        * <b>Secure naming</b>: https://istio.io/docs/concepts/security/#secure-naming
    * <b>Authentication architecture</b>
        * The mesh operator uses .yaml files to specify the policies. The policies are saved in the `Istio configuration storage` once deployed. 
        * <b>Pilot</b> the Istio controller, watches the configuration storage. Upon any policy changes, Pilot translates the new policy to the appropriate configuration telling the <b>Envoy sidecar proxy</b> how to perform the required authentication mechanisms. 
        * <b>Pilot</b> may fetch the public key and attach it to the configuration for JWT validation. 
        * <b>Pilot</b> provides the path to the keys and certificates the Istio system manages and installs them to the application pod for mutual TLS.
        * <b>Envoy proxy</b> receives the configuration, the new authentication requirement takes effect immediately on that pod.
        * <img src="../../images/javaee/authn.svg" width="400px">
    * <b><i>Authentication policies</i></b>: https://istio.io/docs/concepts/security/#authentication-policies
* <b>Authorization</b>
    * Istio’s authorization feature - also known as Role-based Access Control (RBAC) - provides namespace-level, service-level, and method-level access control for services in an Istio Mesh. It features:
        * Role-Based semantics, which are simple and easy to use.
        * Service-to-service and end-user-to-service authorization.
        * Flexibility through custom properties support, for example conditions, in roles and role-bindings.
        * High performance, as Istio authorization is enforced natively on Envoy.
    * <b>Authorization architecture</b>
        *  Operators specify Istio authorization policies using .yaml files. Once deployed, Istio saves the policies in the `Istio Config Store`.
        * <b>Pilot</b> watches for changes to Istio authorization policies. It fetches the updated authorization policies if it sees any changes. 
        * <b>Pilot</b> distributes Istio authorization policies to the <b>Envoy sidecar proxies</b> that are co-located with the service instances.
        * <b>Envoy proxy</b> runs an authorization engine that authorizes requests at runtime. When a request comes to the proxy, the authorization engine evaluates the request context against the current authorization policies, and returns the authorization result, ALLOW or DENY.
        * <img src="../../images/javaee/authz.svg" width="400px">
    * <b><i>Authorization policy</i></b>: https://istio.io/docs/concepts/security/#authorization-policy
* <b><i>Operations</i></b>: https://istio.io/docs/tasks/security/

****

# Observability: Policies and Telemetry
* <b>Infrastructure backends are designed to provide support functionality used to build services. They include such things as access control systems, telemetry capturing systems, quota enforcement systems, billing systems, and so forth.</b>

### Mixer
* <b>Mixer</b> is the Istio component responsible for providing policy controls and telemetry collection:
    * <b>Backend Abstraction</b>: Mixer insulates the rest of Istio from the implementation details of individual infrastructure backends.
    * <b>Intermediation</b>: Mixer allows operators to have fine-grained control over all interactions between the mesh and infrastructure backends.
    * <b>Mixer’s</b> flexibility in dealing with different infrastructure backends comes from its general-purpose plug-in model. <b>`Individual plug-ins are known as adapters`</b> and they allow Mixer to interface to different infrastructure backends that deliver core functionality, such as logging, monitoring, quotas, ACL checking, and more.
    * <img src="../../images/javaee/topology-with-cache.svg" width="400px">
    * <b>Envoy sidecar</b> logically calls Mixer before each request to perform precondition checks, and after each request to report telemetry. The sidecar has `local caching such that a large percentage of precondition checks can be performed from cache`. The sidecar `buffers outgoing telemetry` such that it only calls Mixer infrequently.
        * <b>The sidecar proxies</b> that sit next to each service instance in the mesh must necessarily be frugal in terms of memory consumption, which constrains the possible amount of local caching and buffering. 
    * <b>Mixer</b> lives independently and can use considerably larger caches and output buffers. Mixer thus acts as a `highly-scaled and highly-available second-level cache` for the sidecars.
        * <b>Mixer’s</b> local caches and buffers not only contribute to reduce latency, they also help mask infrastructure backend failures by being able to continue operating even when a backend has become unresponsive.
        * <b>Mixer’s</b> caching and buffering helps reduce the frequency of calls to backends, and can sometimes reduce the amount of data sent to backends.
* <b>Reliability and latency</b>
    * <b>Statelessness</b>: Mixer is stateless in that it doesn’t manage any persistent storage of its own.
    * <b>Hardening</b> Mixer proper is designed to be a highly reliable component. The design intent is to achieve > 99.999% uptime for any individual Mixer instance.
    * <b>Caching and Buffering</b> Mixer is designed to accumulate a large amount of transient ephemeral state.
* <b>Attributes</b>
    * Attributes are an essential concept to Istio’s policy and telemetry functionality. An attribute is a small bit of data that describes a single property of a specific service request or the environment for the request. 
    * Each attribute has a name and a type. The type defines the kind of data that the attribute holds. 
    * <b>Mixer</b> is in essence an attribute processing machine. <b>The Envoy sidecar</b> invokes <b>Mixer</b> for every request, giving Mixer a set of attributes that describe the request and the environment around the request. Based on its configuration and the specific set of attributes it was given, Mixer generates calls to a variety of infrastructure backends.
* <b><i>Configuration model</i></b>: https://istio.io/docs/concepts/policies-and-telemetry/#configuration-model
* <b><i>Operations</i></b>: 
    * https://istio.io/docs/tasks/policy-enforcement/
    * https://istio.io/docs/tasks/telemetry/distributed-tracing/